In [2]:
import sys
print("Python Version:", sys.version)
import statistics
import math
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 500)
%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set()


import datetime
import re
import string
import nltk
import sklearn
from nltk.tokenize import MWETokenizer 
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import PCA
    #py file in directory
from py_files.cleaner import mr_clean_and_tokenize
from py_files.vectorizer import Vectorizer_I_Hardly_Know_Her


pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings('ignore')

Python Version: 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 18:42:56) 
[Clang 10.0.1 ]


In [3]:
#############################################################
#Data graciously taken from https://www.thetrumparchive.com/#
#############################################################
df_tweets = pd.read_csv('tweet_data/trump_tweets_raw.csv')
pd.to_datetime(df_tweets['date'])
df_tweets.sort_values('date',ascending = False, inplace = True,ignore_index = True)
df_tweets=df_tweets.rename(columns = {'date':'DateTime'})
df_tweets['DateTime'] = [datetime.datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df_tweets["DateTime"]]
df_tweets['Date'] = [datetime.datetime.date(d) for d in df_tweets['DateTime']] 
df_tweets['Time'] = [datetime.datetime.time(d) for d in df_tweets['DateTime']] 
#The Day trump became president
mask = (df_tweets['Date'] >= datetime.date(2017, 1, 20))
df_trump = df_tweets[mask]
len(df_trump)

24110

In [5]:
#This function found in py_files/cleaner cleans the tweets
df_trump['tokenized_text'] = df_trump['text'].apply(mr_clean_and_tokenize)
df_trump.head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,DateTime,Date,Time,tokenized_text
0,1320096006038380544,Just landed in Ohio. See you in a little while!,f,f,Twitter for iPhone,37662,4948,2020-10-24 20:13:08,2020-10-24,20:13:08,"[landed, ohio, see, little]"
1,1320095628546899968,Nobody is showing up for Obama’s hate laced sp...,f,f,Twitter for iPhone,57061,10961,2020-10-24 20:11:38,2020-10-24,20:11:38,"[nobody, showing, obama, hate, laced, speeches..."
2,1320076630065373184,AMERICA FIRST!,f,f,Twitter for iPhone,89661,14937,2020-10-24 18:56:09,2020-10-24,18:56:09,[america_first]
3,1320076502839459842,MAKE AMERICA GREAT AGAIN!,f,f,Twitter for iPhone,152417,24873,2020-10-24 18:55:38,2020-10-24,18:55:38,[make_america_great_again]
4,1320076289034850306,Joe Biden = Biggest Tax Increase In History an...,f,f,Twitter for iPhone,57012,11744,2020-10-24 18:54:47,2020-10-24,18:54:47,"[joe, biden, biggest, tax, increase, history, ..."


In [6]:
#Now we can create a count vectorizer from the tokenized column.  #
#This function also filters out sparse words                      #
#Right now the function says a word has to appear 15 times atleast#
###################################################################

df_vectorized = Vectorizer_I_Hardly_Know_Her(df_trump['tokenized_text'])
df_vectorized.head(10)

,abc,abe,abeshinzo,able,abolish,absentee,absolute,absolutely,abuse,accept,...,years,yes,yesterday,yet,york,young,zelensky,zero,zone,zones
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#Let's check out some PCA analysis
pca = PCA(n_components=2)
pca.fit(df_vectorized)
pca_2 = pd.DataFrame(pca.transform(df_vectorized))

In [11]:
#For some reason this is making my computer very slow
plt.scatter(pca_2[0],pca_2[1])
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('First PCA Analysis')

Text(0.5, 1.0, 'First PCA Analysis')

In [10]:
#Unfortunately this few components don't seem to be explaining much variance.
pca.explained_variance_ratio_

array([0.01864724, 0.01404921])